# PoC Scrapping

In [15]:
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)

## Desde get_games traemos:

In [ ]:


# Nombre del archivo de texto que contiene el HTML
file_name = "../data/external/elemento_html_partidos_temporada_24_25.txt"

# Diccionario para almacenar los datos de los partidos
match_data = {}
try:
    # 1. Leer el contenido del archivo .txt
    with open(file_name, "r", encoding="utf-8") as f:
        html_content = f.read()

    # 2. Crear un objeto BeautifulSoup para parsear el HTML
    soup = BeautifulSoup(html_content, "html.parser")
    
    # 3. Buscar todas las filas de la tabla (<tr>)
    filas_partidos = soup.find_all("tr", role="row")
    
    print(f"Extrayendo datos de {len(filas_partidos)} partidos...")

    for fila in filas_partidos:
        # Extraer los datos de las celdas (<td>) de cada fila
        celdas = fila.find_all("td")

        # Asegurarse de que la fila tiene la estructura esperada
        if len(celdas) > 8:
            # Extraer los datos por su índice de celda
            fecha_hora = celdas[0].get_text(strip=True)[-17:]
            nombre_local = celdas[1].get_text(strip=True)
            puntos_local = celdas[3].get_text(strip=True)
            puntos_visita = celdas[4].get_text(strip=True)
            nombre_visita = celdas[6].get_text(strip=True)
            
            # El link está dentro de la celda en el índice 8 (anteriormente 9)
            link_tag = celdas[8].find("a", href=True)
            link_estadisticas = link_tag.get('href') if link_tag else None
            
            # Usar una combinación única como clave del diccionario
            match_key = f"{nombre_local} vs {nombre_visita} ({fecha_hora})"
            
            # Guardar los datos en el diccionario
            match_data[match_key] = {
                "nombre_local": nombre_local,
                "puntos_local": puntos_local,
                "nombre_visita": nombre_visita,
                "puntos_visita": puntos_visita,
                "link_estadisticas": link_estadisticas
            }
    
    print("\nDiccionario de datos de partidos creado.")
    
    # Mostrar el diccionario para su verificación
    display(match_data)
    
except FileNotFoundError:
    print(f"Error: No se encontró el archivo '{file_name}'. Asegúrate de que el archivo existe en la ruta correcta.")
except Exception as e:
    print(f"Ocurrió un error al procesar el archivo: {e}")

Extrayendo datos de 380 partidos...

Diccionario de datos de partidos creado.


{'ATENAS (C) vs BOCA (007/10/2024 22:10)': {'nombre_local': 'ATENAS (C)',
  'puntos_local': '69',
  'nombre_visita': 'BOCA',
  'puntos_visita': '81',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1'},
 'OBRAS vs PLATENSE (008/10/2024 20:00)': {'nombre_local': 'OBRAS',
  'puntos_local': '94',
  'nombre_visita': 'PLATENSE',
  'puntos_visita': '90',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/UYiBwuNnKVL9qCbXhOY76g==?a=1'},
 'INSTITUTO vs OLIMPICO (LB) (008/10/2024 21:00)': {'nombre_local': 'INSTITUTO',
  'puntos_local': '99',
  'nombre_visita': 'OLIMPICO (LB)',
  'puntos_visita': '73',
  'link_estadisticas': 'https://estadisticascabb.gesdeportiva.es/partido/Dwgh5U3-8hNWcMtXCbn6yw==?a=1'},
 'UNION (SF) vs SAN LORENZO (008/10/2024 21:30)': {'nombre_local': 'UNION (SF)',
  'puntos_local': '68',
  'nombre_visita': 'SAN LORENZO',
  'puntos_visita': '63',
  'link_estadisticas': 'https://estadisticascabb.gesdepor

## Pruebas de web scrapping

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd

### Mapa de tiro

In [8]:
# Nota: Este script asume que la variable 'match_data' ya fue creada
#       y contiene los datos de los partidos, incluyendo los enlaces.
#       Por ejemplo:
# match_data = {
#     "ATENAS (C) vs BOCA (07/10/2024 22:10)": {
#         "nombre_local": "ATENAS (C)",
#         "puntos_local": "69",
#         "nombre_visita": "BOCA",
#         "puntos_visita": "81",
#         "link_estadisticas": "https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1"
#     }
#     ...
# }

# --- Parte 1: Obtener el primer enlace del diccionario `match_data` ---
try:
    if not match_data:
        raise ValueError("El diccionario 'match_data' está vacío o no ha sido creado. No se puede continuar.")
        
    # Obtener el primer enlace del diccionario
    primer_partido_key = list(match_data.keys())[0]
    primer_link = match_data[primer_partido_key]["link_estadisticas"]
    print(f"El enlace del primer partido es: {primer_link}")

except Exception as e:
    print(f"Ocurrió un error al obtener el enlace del diccionario: {e}")
    primer_link = None

# --- Parte 2: Usar Selenium para navegar a ese enlace y extraer datos del mapa de tiro ---
if primer_link:
    try:
        print("\nIniciando Selenium para navegar al mapa de tiro...")
        
        # Configuración de Selenium
        options = Options()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        # options.add_argument("--headless")  # Descomenta si no quieres ver el navegador
        
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        driver.get(primer_link)

        # Esperar el iframe principal y cambiar a él
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        
        # Ahora, dentro del iframe principal, buscar el iframe del mapa de tiro
        iframe_mapa = WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "iframe[src*='mapa-tiro']"))
        )
        driver.switch_to.frame(iframe_mapa)
        
        # Extraer el HTML del iframe del mapa de tiro
        html_mapa_tiro = driver.page_source
        
        # Procesar el HTML con BeautifulSoup
        soup = BeautifulSoup(html_mapa_tiro, "html.parser")
        tiros = soup.find_all("i", class_="ico-tiro")
        
        print(f"Se encontraron {len(tiros)} tiros en el mapa del primer partido.")

        # Crear un DataFrame con los datos de los tiros
        data = []
        for tiro in tiros:
            clases = tiro.get("class", [])
            estilo = tiro.get("style", "")
            
            resultado = "fallado" if "fa-times" in clases else "acertado" if "fa-circle" in clases else "desconocido"
            equipo = "azul" if "colorAZUL" in clases else "rojo" if "colorROJO" in clases else "desconocido"
            
            try:
                left = float(estilo.split("left:")[1].split("%")[0].strip())
                top = float(estilo.split("top:")[1].split("%")[0].strip())
            except (IndexError, ValueError):
                left, top = None, None
            
            data.append({
                "equipo": equipo,
                "resultado": resultado,
                "left_pct": left,
                "top_pct": top
            })

        df_tiros = pd.DataFrame(data)
        print("\nDataFrame de tiros del primer partido:")
        display(df_tiros.head(20))

    except Exception as e:
        print(f"Ocurrió un error en la parte de Selenium: {e}")
    finally:
        try:
            driver.quit()
        except:
            pass

El enlace del primer partido es: https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1

Iniciando Selenium para navegar al mapa de tiro...
Se encontraron 121 tiros en el mapa del primer partido.

DataFrame de tiros del primer partido:


,equipo,resultado,left_pct,top_pct
0,azul,fallado,62.79,24.70
1,desconocido,acertado,14.51,26.88
2,azul,fallado,84.91,28.09
3,azul,fallado,62.21,26.15
4,desconocido,acertado,15.37,27.36
5,azul,acertado,88.51,34.38
6,desconocido,fallado,15.95,25.67
7,azul,fallado,67.82,20.34
8,desconocido,acertado,16.67,26.39
9,azul,fallado,87.93,28.57


### Box Score

In [13]:
# Importaciones necesarias
import pandas as pd
import re
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time

# --- Datos de ejemplo ---
match_data = {
    "ATENAS (C) vs BOCA (07/10/2024 22:10)": {
        "link_estadisticas": "https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1"
    }
}
# --- Fin del ejemplo ---

all_player_stats = []
driver = None

try:
    primer_partido_key = list(match_data.keys())[0]
    primer_link = match_data[primer_partido_key]["link_estadisticas"]
    print(f"El enlace del partido es: {primer_link}")

    # --- Parte 1: NAVEGACIÓN CON SELENIUM (YA FUNCIONA CORRECTAMENTE) ---
    print("\nIniciando Selenium...")
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.get(primer_link)
    
    wait = WebDriverWait(driver, 20)

    print("Cambiando al iframe principal...")
    wait.until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    print("Cambio al iframe principal exitoso.")

    print("Buscando y haciendo clic en la pestaña 'Estadísticas'...")
    tab_selector = (By.CSS_SELECTOR, "li.pestana-estadisticas")
    estadisticas_tab_element = wait.until(EC.visibility_of_element_located(tab_selector))
    driver.execute_script("arguments[0].click();", estadisticas_tab_element)
    time.sleep(1)

    print("Cambiando al iframe de datos de estadísticas...")
    iframe_de_datos_selector = (By.CSS_SELECTOR, "div.contenido-estadisticas.activo iframe")
    wait.until(EC.frame_to_be_available_and_switch_to_it(iframe_de_datos_selector))
    print("Acceso al iframe de datos exitoso.")

    print("Extrayendo el código HTML final...")
    html_box_scores = driver.page_source
    
    # --- Parte 2: EXTRACCIÓN CON BEAUTIFULSOUP (SECCIÓN CORREGIDA) ---
    print("Procesando el HTML para extraer datos de jugadores...")
    soup = BeautifulSoup(html_box_scores, "html.parser")
    
    # --- INICIO DE LA CORRECCIÓN ---
    # Hacemos la búsqueda de los nombres de equipo de forma segura
    
    equipo_local_nombre = "Local" # Valor por defecto
    equipo_visitante_nombre = "Visitante" # Valor por defecto
    
    # Buscamos los divs que contienen los nombres de los equipos.
    # El selector 'div.nombre-equipo' es un candidato común.
    divs_nombres = soup.find_all("div", class_="nombre-equipo")
    if len(divs_nombres) >= 2:
        equipo_local_nombre = divs_nombres[0].get_text(strip=True)
        equipo_visitante_nombre = divs_nombres[1].get_text(strip=True)
    else:
        print("Advertencia: No se encontraron los nombres de equipo con el selector 'div.nombre-equipo'. Se usarán nombres genéricos.")
    # --- FIN DE LA CORRECCIÓN ---

    tablas = soup.find_all("table", class_="tabla-estadisticas")
    
    if len(tablas) >= 2:
        print(f"\nExtrayendo datos de: {equipo_local_nombre}")
        for fila in tablas[0].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_local_nombre
                all_player_stats.append(player_data)

        print(f"Extrayendo datos de: {equipo_visitante_nombre}")
        for fila in tablas[1].find("tbody").find_all("tr", onclick=True):
            onclick_attr = fila["onclick"]
            match = re.search(r"(\{.*\})", onclick_attr)
            if match:
                json_str = match.group(1).replace("'", '"')
                player_data = json.loads(json_str)
                player_data['equipo'] = equipo_visitante_nombre
                all_player_stats.append(player_data)
    
    # --- Parte 3: PRESENTACIÓN DE DATOS CON PANDAS ---
    if all_player_stats:
        df_box_scores = pd.DataFrame(all_player_stats)
        print("\n✅ DataFrame final con todos los datos extraídos:")
        df_box_scores
    else:
        print("\nNo se pudieron extraer datos de jugadores del HTML.")

except Exception as e:
    print(f"Ocurrió un error general: {e}")
finally:
    if driver:
        driver.quit()
        print("\nNavegador cerrado.")


El enlace del partido es: https://estadisticascabb.gesdeportiva.es/partido/rM2-eTJQHJsR2FLJYE8GRw==?a=1

Iniciando Selenium...
Cambiando al iframe principal...
Cambio al iframe principal exitoso.
Buscando y haciendo clic en la pestaña 'Estadísticas'...
Cambiando al iframe de datos de estadísticas...
Acceso al iframe de datos exitoso.
Extrayendo el código HTML final...
Procesando el HTML para extraer datos de jugadores...

Extrayendo datos de: ATENAS (C)
Extrayendo datos de: BOCA

✅ DataFrame final con todos los datos extraídos:

Navegador cerrado.


In [18]:
df_box_scores.columns

Index(['IdJugador', 'IdClub', 'IdEquipo', 'Dorsal', 'Nombre', 'NombreCompleto',
       'Partidos', 'Minutos', 'MinutosPorPartido', 'Puntos',
       'PuntosPorPartido', 'TirosDos', 'TirosTres', 'TirosLibres',
       'ReboteDefensivo', 'ReboteDefensivoPorPartido', 'ReboteOfensivo',
       'ReboteOfensivoPorPartido', 'RebotesTotales',
       'RebotesTotalesPorPartido', 'Asistencias', 'AsistenciasPorPartido',
       'Recuperaciones', 'RecuperacionesPorPartido', 'Perdidas',
       'PerdidasPorPartido', 'TaponCometido', 'TaponCometidoPorPartido',
       'TaponRecibido', 'TaponRecibidoPorPartido', 'FaltaCometida',
       'FaltaCometidaPorPartido', 'FaltaRecibida', 'FaltaRecibidaPorPartido',
       'Valoracion', 'ValoracionPorPartido', 'TiempoJuego', 'CincoInicial',
       'MinutosTabla', 'MinutosPorPartidoTabla', 'PuntosPorPartidoTabla',
       'ReboteDefensivoPorPartidoTabla', 'ReboteOfensivoPorPartidoTabla',
       'RebotesTotalesPorPartidoTabla', 'AsistenciasPorPartidoTabla',
       'Recup

In [16]:
df_box_scores #TODO revisar datos

,IdJugador,IdClub,IdEquipo,Dorsal,Nombre,NombreCompleto,Partidos,Minutos,MinutosPorPartido,Puntos,PuntosPorPartido,TirosDos,TirosTres,TirosLibres,ReboteDefensivo,ReboteDefensivoPorPartido,ReboteOfensivo,ReboteOfensivoPorPartido,RebotesTotales,RebotesTotalesPorPartido,Asistencias,AsistenciasPorPartido,Recuperaciones,RecuperacionesPorPartido,Perdidas,PerdidasPorPartido,TaponCometido,TaponCometidoPorPartido,TaponRecibido,TaponRecibidoPorPartido,FaltaCometida,FaltaCometidaPorPartido,FaltaRecibida,FaltaRecibidaPorPartido,Valoracion,ValoracionPorPartido,TiempoJuego,CincoInicial,MinutosTabla,MinutosPorPartidoTabla,PuntosPorPartidoTabla,ReboteDefensivoPorPartidoTabla,ReboteOfensivoPorPartidoTabla,RebotesTotalesPorPartidoTabla,AsistenciasPorPartidoTabla,RecuperacionesPorPartidoTabla,PerdidasPorPartidoTabla,TaponCometidoPorPartidoTabla,TaponRecibidoPorPartidoTabla,FaltaCometidaPorPartidoTabla,FaltaRecibidaPorPartidoTabla,ValoracionPorPartidoTabla,equipo
0,78377,1498,0,2,"ARAUJO, M.","ARAUJO, MAXIMO",0,0,0,3,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,18:05,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
1,326699,1498,0,4,"BUENDIA, C.","BUENDIA, CARLOS MANUEL",0,0,0,1,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,3,0,1,0,07:57,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
2,273565,1498,0,10,"MONTERO, J.","MONTERO, JOSE IGNACIO",0,0,0,2,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 2, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,3,0,1,0,2,0,0,0,0,0,0,0,2,0,4,0,23:27,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
3,209515,1498,0,13,"ARN BUSTAMANTE, L.","ARN BUSTAMANTE, LUCAS MARTIN",0,0,0,12,0,"{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 3, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...",1,0,0,0,1,0,4,0,0,0,0,0,0,0,0,0,2,0,3,0,9,0,30:14,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
4,321117,1498,0,17,"MARCONETTI, S.","MARCONETTI, SANTIAGO JAVIER",0,0,0,0,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
5,209521,1498,0,20,"LEMA, L.","LEMA, LEONARDO",0,0,0,10,0,"{'Aciertos': 2, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 3, 'AciertosPorPartido': 0, 'Fall...",7,0,0,0,7,0,2,0,1,0,1,0,0,0,1,0,3,0,5,0,13,0,36:50,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
6,235314,1498,0,21,"VALFRE, F.","VALFRE, FACUNDO NICOLAS",0,0,0,0,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
7,171753,1498,0,22,"MAIDANA, J.","MAIDANA, JERONIMO",0,0,0,6,0,"{'Aciertos': 3, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...",2,0,1,0,3,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,6,0,12:56,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
8,326727,1498,0,31,"BERNABEI, L.","BERNABEI, LISANDRO",0,0,0,0,0,"{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 0, 'AciertosPorPartido': 0, 'Fall...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,00:00,False,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ATENAS (C)
9,276607,1498,0,32,"BUEMO, C.","BUEMO, CARLOS EMANUEL",0,0,0,15,0,"{'Aciertos': 6, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos': 1, 'AciertosPorPartido': 0, 'Fall...","{'Aciertos'

In [ ]:
# --- Parte 4: Usar Selenium para navegar a ese enlace y extraer datos del play-by-play ---
if primer_link:
    try:
        print("\nIniciando Selenium para navegar al play-by-play...")
        
        # Configuración de Selenium
        options = Options()
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        options.add_argument("--headless")
        
        # Forzar la descarga del driver para asegurar compatibilidad
        service = Service(ChromeDriverManager().install(force_download=True))
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(primer_link)

        # Esperar el iframe principal y cambiar a él
        WebDriverWait(driver, 20).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        
        # Buscar el iframe específico del play-by-play (se asume que es el único)
        WebDriverWait(driver, 20).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "ul.listadoAccionesPartido"))
        )
        
        # Extraer el HTML del iframe
        html_play_by_play = driver.page_source
        
        # Procesar el HTML con BeautifulSoup
        soup = BeautifulSoup(html_play_by_play, "html.parser")
        
        acciones = []
        
        listado_acciones = soup.find_all("li", class_=["accion_local", "accion_visita"])

        print(f"\nExtrayendo datos de {len(listado_acciones)} acciones del play-by-play...")

        for accion in listado_acciones:
            equipo = "local" if "accion_local" in accion.get("class", []) else "visitante"
            
            # Extraer tiempo
            tiempo_tag = accion.find("div", class_="tiempo")
            tiempo = tiempo_tag.get_text(strip=True) if tiempo_tag else "N/A"
            
            # Extraer tipo de acción y jugador
            info_tag = accion.find("div", class_="informacion")
            tipo_accion = info_tag.find("strong").get_text(strip=True) if info_tag and info_tag.find("strong") else "N/A"
            jugador_tag = info_tag.find("span")
            jugador = jugador_tag.get_text(strip=True) if jugador_tag else "N/A"

            # Extraer puntuación (si existe, ya que no todas las acciones tienen)
            puntos_tag = accion.find("td", class_="puntos_locales") or accion.find("td", class_="puntos_visitantes")
            puntos = puntos_tag.get_text(strip=True) if puntos_tag else "N/A"
            
            acciones.append({
                "equipo": equipo,
                "tiempo": tiempo,
                "tipo_accion": tipo_accion,
                "jugador": jugador,
                "puntos_equipo": puntos,
            })

        df_play_by_play = pd.DataFrame(acciones)
        
        print("\nDataFrame con las acciones del play-by-play:")
        display(df_play_by_play.head(20))

    except Exception as e:
        print(f"Ocurrió un error en la parte de Selenium o BeautifulSoup: {e}")
        try:
            driver.quit()
        except:
            pass


Iniciando Selenium para navegar al play-by-play...
Ocurrió un error en la parte de Selenium o BeautifulSoup: ChromeDriverManager.install() got an unexpected keyword argument 'force_download'


In [7]:
df_play_by_play

NameError: name 'df_play_by_play' is not defined